# Data

In [1]:
from textwrap import dedent
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from datasets import load_dataset

dataset = load_dataset("yqzheng/semeval2014_restaurants")

In [3]:
import pandas as pd
train = pd.DataFrame(dataset['train'])
test = pd.DataFrame(dataset['test'])
train.shape, test.shape

((3608, 5), (1120, 5))

In [4]:
train.columns

Index(['text', 'aspect', 'start', 'end', 'label'], dtype='object')

# Instructor

## Data

In [158]:
def create_list(df):
    df['json'] = df.apply(lambda row: {'aspect': row['aspect'], 'label': row['label']} , axis=1)
    return df.groupby('text')['json'].agg(list).reset_index()

In [159]:
train_list = create_list(train)
test_list = create_list(test)
train_list.head(10)

,text,json
0,"$160 for 2 filets, 2 sides, an appetizer and d...","[{'aspect': 'filets', 'label': 0}, {'aspect': ..."
1,$20 for all you can eat sushi cannot be beaten.,"[{'aspect': 'sushi', 'label': 0}]"
2,$20 gets you unlimited sushi of a very high qu...,"[{'aspect': 'sushi', 'label': 1}, {'aspect': '..."
3,"$6 and there is much tasty food, all of it fre...","[{'aspect': 'food', 'label': 1}]"
4,"($200 for 2 glasses of champagne, not too expe...","[{'aspect': 'glasses of champagne', 'label': -..."
5,(Always ask the bartender for the SEASONAL bee...,"[{'aspect': 'SEASONAL beer', 'label': 1}, {'as..."
6,(and I have eaten my share) Which impresses me...,"[{'aspect': 'serve', 'label': 1}]"
7,"(food was delivered by a busboy, not waiter) W...","[{'aspect': 'food', 'label': 0}, {'aspect': 'b..."
8,- the bread at the beginning is super tasty an...,"[{'aspect': 'bread', 'label': 1}, {'aspect': '..."
9,20 minutes for our reservation but it gave us ...,"[{'aspect': 'reservation', 'label': -1}, {'asp..."


## Model

In [30]:
import instructor
from pydantic import BaseModel
from openai import OpenAI
from dotenv import load_dotenv
from textwrap import dedent
import os

load_dotenv()

True

In [58]:
def use_instructor(review: str, model: str="gpt-3.5-turbo"):
    class AspectInfo(BaseModel):
        aspect: str
        label: int

    example_review = train_df.loc[5, 'text']
    example_label = train_df.loc[5, 'json']
    prompt = dedent("""
                Please help me extract the aspects in the following customer review, and for each aspect, 
                label it as 1 for positive, 0 for neutral, or -1 for negative.

                Customer Review: Always ask the bartender for the SEASONAL beer!!!

                AspectInfo: [{'aspect': 'SEASONAL beer', 'label': 1}, {'aspect': 'bartender', 'label': 0}]

                Customer Review: 
                """)

    client = instructor.from_openai(OpenAI())
    aspects = client.chat.completions.create_iterable(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt},
        ],
        response_model=AspectInfo,
    )
    return [{"aspect": aspect.aspect, "label": aspect.label} for aspect in aspects]

In [59]:
example_review = train_df.loc[5, 'text']
example_label = train_df.loc[5, 'json']
test_review = test_df.loc[4, 'text']
test_label = test_df.loc[4, 'json']

print(example_review)
print(example_label)
print(test_review)
print(test_label)

(Always ask the bartender for the SEASONAL beer!!!
[{'aspect': 'SEASONAL beer', 'label': 1}, {'aspect': 'bartender', 'label': 0}]
A beautiful atmosphere, perfect for drinks and/or appetizers.
[{'aspect': 'atmosphere', 'label': 1}, {'aspect': 'drinks', 'label': 0}, {'aspect': 'appetizers', 'label': 0}]


In [60]:
res = use_instructor(test_review)
res

[{'aspect': 'SEASONAL beer', 'label': 1}, {'aspect': 'bartender', 'label': 0}]

In [56]:
import json

comp = [{'aspect': 'bartender', 'label': 0}, {'aspect': 'SEASONAL beer', 'label': 1}]

sorted(comp, key=lambda x: json.dumps(x, sort_keys=True)) == sorted(res, key=lambda x: json.dumps(x, sort_keys=True))

True

In [101]:
from pydantic import BaseModel, Field, ConfigDict
from typing import List, Iterable

class AspectInfo(BaseModel):
    aspect: str
    label: int

    model_config = ConfigDict(
            json_schema_extra={
                "examples": [json for i in range(5) for json in train_df.loc[i, 'json']]
            }
    )

In [111]:
prompt = dedent(f"""
                Please help me extract the aspects in the following customer review, and for each aspect, 
                label it as 1 for positive, 0 for neutral, or -1 for negative. The response should be a valid Python
                list of JSON objects, and each JSON object contains an aspect and a label. 

                Customer Review: {example_review}

                Entities: {example_label}

                Customer Review: {test_review}
                """)

In [117]:
client = instructor.from_openai(OpenAI())

# client = Groq(
#     api_key=os.environ.get("GROQ_API_KEY"),
# )

# client = instructor.from_groq(client, mode=instructor.Mode.JSON)

In [121]:
aspects = client.chat.completions.create(
    model="gpt-3.5-turbo",
    # model="llama3-70b-8192",
    messages=[
        {"role": "user", "content": prompt},
    ],
    response_model=Iterable[AspectInfo],
)

In [122]:
aspects

[AspectInfo(aspect='atmosphere', label=1),
 AspectInfo(aspect='drinks', label=1),
 AspectInfo(aspect='appetizers', label=1)]

# DSPY

## DSPY Tracer

In [5]:
import phoenix as px
phoenix_session = px.launch_app()

from openinference.instrumentation.dspy import DSPyInstrumentor
from opentelemetry import trace as trace_api
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
from opentelemetry.sdk import trace as trace_sdk
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace.export import SimpleSpanProcessor

endpoint = "http://127.0.0.1:6006/v1/traces"
resource = Resource(attributes={})
tracer_provider = trace_sdk.TracerProvider(resource=resource)
span_otlp_exporter = OTLPSpanExporter(endpoint=endpoint)
tracer_provider.add_span_processor(SimpleSpanProcessor(span_exporter=span_otlp_exporter))

trace_api.set_tracer_provider(tracer_provider=tracer_provider)
DSPyInstrumentor().instrument()

🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


## Data

In [6]:
def merge_jsons(json_list):
    result = {}
    for j in json_list:
        result.update(j)
    return result

def create_json(df):
    df['json'] = df.apply(lambda row: {row['aspect']: row['label']} , axis=1)
    return df.groupby('text')['json'].agg(merge_jsons).reset_index()

In [7]:
train_json = create_json(train)
test_json = create_json(test)
train_json.head(10)

,text,json
0,"$160 for 2 filets, 2 sides, an appetizer and d...","{'filets': 0, 'sides': 0, 'appetizer': 0, 'dri..."
1,$20 for all you can eat sushi cannot be beaten.,{'sushi': 0}
2,$20 gets you unlimited sushi of a very high qu...,"{'sushi': 1, 'sushi places': 1, 'quality': 1}"
3,"$6 and there is much tasty food, all of it fre...",{'food': 1}
4,"($200 for 2 glasses of champagne, not too expe...","{'glasses of champagne': -1, 'bottle of wine':..."
5,(Always ask the bartender for the SEASONAL bee...,"{'SEASONAL beer': 1, 'bartender': 0}"
6,(and I have eaten my share) Which impresses me...,{'serve': 1}
7,"(food was delivered by a busboy, not waiter) W...","{'food': 0, 'busboy': -1, 'waiter': -1, 'chees..."
8,- the bread at the beginning is super tasty an...,"{'bread': 1, 'pizza': 1, 'margarite pizza with..."
9,20 minutes for our reservation but it gave us ...,"{'reservation': -1, 'cocktails': 1, 'surroundi..."


In [8]:
import dspy

gpt = dspy.OpenAI(model="gpt-3.5-turbo", max_tokens=1000)
dspy.settings.configure(lm=gpt)

In [9]:
dspy_train = [dspy.Example(review=row['text'], aspects_with_label=row['json'], aspects_list=str(row['json'].keys())).with_inputs('review') for _, row in train_json.iterrows()]
dspy_test = [dspy.Example(review=row['text'], aspects_with_label=row['json'], aspects_list=str(row['json'].keys())).with_inputs('review') for _, row in test_json.iterrows()]

In [10]:
from dspy.evaluate.evaluate import Evaluate

evaluate_test = Evaluate(devset=dspy_test[:100], num_threads=10, display_progress=True, display_table=5, max_errors=99999)

In [11]:
def validate_venn(example, pred, trace=None):
    total_count = len(merge_jsons([example.aspects_with_label, pred.aspects_with_label]).keys())
    accurate_count = sum(1 for k, v in example.aspects_with_label.items() if pred.aspects_with_label.get(k) == v)
    if trace is None:
        return float(accurate_count) / total_count
    else:
        return accurate_count > total_count * 0.8

In [12]:
def validate_f1(example, pred, trace=None):
    tp = sum(1 for k, v in example.aspects_with_label.items() if pred.aspects_with_label.get(k) == v)
    fp = sum(1 for k, v in pred.aspects_with_label.items() if example.aspects_with_label.get(k) != v)
    fn = sum(1 for k, v in example.aspects_with_label.items() if k not in pred.aspects_with_label.keys())
    if (0.5 * (fp + fn) + tp) == 0:
        return 0
    f1 = float(tp) / (0.5 * (fp + fn) + tp)
    if trace is None:
        return f1
    else:
        return f1 >= 0.8

# def validate_f1(example, pred, trace=None):
#     tp = sum(1 for k, v in example.aspects_with_label.items() if v == 1 and pred.aspects_with_label.get(k) == v)
#     fp = sum(1 for k, v in pred.aspects_with_label.items() if v == 1 and example.aspects_with_label.get(k) != v)
#     fn = sum(1 for k, v in pred.aspects_with_label.items() if v == -1 and example.aspects_with_label.get(k) != v)
#     if (0.5 * (fp + fn) + tp) == 0:
#         return 0
#     f1 = float(tp) / (0.5 * (fp + fn) + tp)
#     if trace is None:
#         return f1
#     else:
#         return f1 >= 0.8

## Basic Model

In [13]:
class Review2Aspects(dspy.Signature):
    """
    Identify aspects and their sentiments from a customer review. The aspects must be words or phrases in the review.
    The response should be a Python dictionary, where each key is an aspect and the value is a sentiment label.
    A label of 1 indicates positive sentiment, 0 indicates neutral sentiment, and -1 indicates negative sentiment.
    """

    review: str = dspy.InputField(desc="a customer review")
    aspects_with_label: dict = dspy.OutputField(format=dict, desc=dedent("""
        a single Python dictionary, where each key is an aspect and the value is the label,
        with label 1 indicating positive sentiment, 0 indicating neutral sentiment, and -1
        indicating negative sentiment.
        """))

In [14]:
dspy_test[0].review

'" The menu includes pub fare--burgers, steaks and shepherds pie--and even a portabella lasagna for those black sheep known as "vegetarians.'

In [15]:
class ABSAPipeline(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_aspects = dspy.TypedChainOfThought(Review2Aspects)

    def forward(self, review):
        pred = self.generate_aspects(review=review)
        return dspy.Prediction(aspects_with_label=pred.aspects_with_label)

In [16]:
uncompiled_pipeline = ABSAPipeline()

In [17]:
pred = uncompiled_pipeline(dspy_test[0].review)

In [18]:
pred

Prediction(
    aspects_with_label={'menu': 1, 'pub fare': 1, 'burgers': 1, 'steaks': 1, 'shepherds pie': 1, 'portabella lasagna': 1, 'vegetarians': 1}
)

In [157]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

optimizer = BootstrapFewShotWithRandomSearch(metric=validate_venn)
compiled_pipeline = optimizer.compile(ABSAPipeline(), teacher=ABSAPipeline(), trainset=dspy_train[:100])

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 16 candidate sets.





  0%|                                                       | 0/100 [00:00<?, ?it/s]


Average Metric: 0.0 / 1  (0.0):   0%|                       | 0/100 [00:00<?, ?it/s]


Average Metric: 0.25 / 2  (12.5):   1%|▏            | 1/100 [00:00<00:08, 12.07it/s]


Average Metric: 0.5 / 3  (16.7):   2%|▎             | 2/100 [00:00<00:04, 21.31it/s]


Average Metric: 0.5 / 4  (12.5):   3%|▍             | 3/100 [00:00<00:03, 26.60it/s]


Average Metric: 0.5 / 4  (12.5):   4%|▌             | 4/100 [00:00<00:02, 32.27it/s]


Average Metric: 0.5 / 5  (10.0):   4%|▌             | 4/100 [00:00<00:02, 32.27it/s]


Average Metric: 0.5 / 6  (8.3):   5%|▊              | 5/100 [00:00<00:02, 32.27it/s]


Average Metric: 0.75 / 7  (10.7):   6%|▊            | 6/100 [00:00<00:02, 32.27it/s]


Average Metric: 1.0833333333333333 / 8  (13.5):   7%| | 7/100 [00:00<00:02, 32.27it/


Average Metric: 1.0833333333333333 / 9  (12.0):   8%| | 8/100 [00:00<00:02, 32.27it/


Average Metric: 1.0833333333333333 / 10 

Average Metric: 30.60555555555555 / 100  (30.6%)
Score: 30.61 for set: [0]
New best score: 30.61 for seed -3
Scores so far: [30.61]
Best score: 30.61





  0%|                                                       | 0/100 [00:00<?, ?it/s]


Average Metric: 0.5 / 1  (50.0):   0%|                      | 0/100 [00:01<?, ?it/s]


Average Metric: 0.5 / 1  (50.0):   1%|▏             | 1/100 [00:01<02:00,  1.22s/it]


Average Metric: 0.5 / 2  (25.0):   1%|▏             | 1/100 [00:01<02:00,  1.22s/it]


Average Metric: 0.5 / 3  (16.7):   2%|▎             | 2/100 [00:01<01:59,  1.22s/it]


Average Metric: 0.5 / 3  (16.7):   3%|▍             | 3/100 [00:02<00:59,  1.63it/s]


Average Metric: 0.8333333333333333 / 4  (20.8):   3%| | 3/100 [00:02<00:59,  1.63it/


Average Metric: 1.8333333333333333 / 5  (36.7):   4%| | 4/100 [00:02<00:58,  1.63it/


Average Metric: 1.8333333333333333 / 6  (30.6):   5%| | 5/100 [00:02<00:58,  1.63it/


Average Metric: 1.8333333333333333 / 6  (30.6):   6%| | 6/100 [00:02<00:31,  2.97it/


Average Metric: 1.8333333333333333 / 7  (26.2):   6%| | 6/100 [00:03<00:31,  2.97it/


Average Metric: 1.8333333333333333 / 7  

Backing off 0.6 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}


...


Average Metric: 52.405555555555544 / 93  (56.4):  92%|▉| 92/100 [01:05<00:09,  1.15s
Average Metric: 52.405555555555544 / 93  (56.4):  93%|▉| 93/100 [01:05<00:09,  1.32s
Average Metric: 53.405555555555544 / 94  (56.8):  93%|▉| 93/100 [01:05<00:09,  1.32s
Average Metric: 53.405555555555544 / 94  (56.8):  94%|▉| 94/100 [01:05<00:06,  1.06s

Backing off 1.9 seconds after 2 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}



Average Metric: 53.405555555555544 / 95  (56.2):  94%|▉| 94/100 [01:06<00:06,  1.06s
Average Metric: 53.405555555555544 / 95  (56.2):  95%|▉| 95/100 [01:06<00:05,  1.07s
Average Metric: 54.405555555555544 / 96  (56.7):  95%|▉| 95/100 [01:08<00:05,  1.07s
Average Metric: 54.405555555555544 / 96  (56.7):  96%|▉| 96/100 [01:08<00:04,  1.15s
Average Metric: 54.405555555555544 / 97  (56.1):  96%|▉| 96/100 [01:10<00:04,  1.15s
Average Metric: 54.405555555555544 / 97  (56.1):  97%|▉| 97/100 [01:10<00:04,  1.40s
Average Metric: 55.405555555555544 / 98  (56.5):  97%|▉| 97/100 [01:11<00:04,  1.40s
Average Metric: 55.405555555555544 / 98  (56.5):  98%|▉| 98/100 [01:11<00:02,  1.31s
Average Metric: 55.405555555555544 / 99  (56.0):  98%|▉| 98/100 [01:12<00:02,  1.31s
Average Metric: 55.405555555555544 / 99  (56.0):  99%|▉| 99/100 [01:12<00:01,  1.38s
Average Metric: 56.405555555555544 / 100  (56.4):  99%|▉| 99/100 [02:48<00:01,  1.38
Average Metric: 56.405555555555544 / 100  (56.4): 100%|█| 100/10

Average Metric: 56.405555555555544 / 100  (56.4%)
Score: 56.41 for set: [16]
Scores so far: [30.61, 57.96, 53.19, 51.3, 54.02, 56.31, 59.32, 54.7, 56.91, 52.12, 53.05, 56.87, 57.03, 55.56, 51.82, 54.14, 53.34, 56.92, 56.41]
Best score: 59.32
Average of max per entry across top 1 scores: 0.5931717171717172
Average of max per entry across top 2 scores: 0.7806666666666666
Average of max per entry across top 3 scores: 0.8848333333333334
Average of max per entry across top 5 scores: 0.9408333333333335
Average of max per entry across top 8 scores: 0.98
Average of max per entry across top 9999 scores: 1.0
19 candidate programs found.



/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


In [158]:
uncompiled_pipeline_score = evaluate_test(uncompiled_pipeline, metric=validate_f1, display=False)

compiled_pipeline_score = evaluate_test(compiled_pipeline, metric=validate_f1)

/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)

Average Metric: 2.25 / 4  (56.2):   3%|▍            | 3/100 [00:01<02:36,  1.61s/it]
Average Metric: 2.5833333333333335 / 5  (51.7):   4%| | 4/100 [00:02<02:34,  1.61s/i
Average Metric: 2.5833333333333335 / 5  (51.7):   5%| | 5/100 [00:02<00:37,  2.55it/
Average Metric: 2.916666666666667 / 6  (48.6):   5%| | 5/100 [00:02<00:37,  2.55it/s
Average Metric: 2.916666666666667 / 6  (48.6):   6%| | 6/100 [00:02<00:32,  2.89it/s
Average Metric: 3.2500000000000004 / 7  (46.4):   6%| | 6/100 [00:02<00:32,  2.89it/
Average Metric: 3.2500000000000004 / 7  (46.4):   7%| | 7/100 [00:02<00:31,  2.99it/
Average Metric: 3.5000000000000004 / 8  (43.8):   7%| | 7/100 [00:03<00:31,  2.99it/
Average Metric: 3.5000000000000004 / 8  (43.8):   8%| | 8/100 [00:03<00:28,  3.20it/
Average Metri

Backing off 0.9 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}



Average Metric: 26.614285714285714 / 47  (56.6):  46%|▍| 46/100 [00:12<00:11,  4.70i
Average Metric: 26.614285714285714 / 47  (56.6):  47%|▍| 47/100 [00:12<00:13,  3.81i
Average Metric: 26.947619047619046 / 48  (56.1):  47%|▍| 47/100 [00:12<00:13,  3.81i
Average Metric: 26.947619047619046 / 48  (56.1):  48%|▍| 48/100 [00:12<00:11,  4.48i

Backing off 0.5 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 0.9 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}



Average Metric: 27.947619047619046 / 49  (57.0):  48%|▍| 48/100 [00:13<00:11,  4.48i
Average Metric: 27.947619047619046 / 49  (57.0):  49%|▍| 49/100 [00:13<00:15,  3.38i

...

Backing off 32.7 seconds after 7 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}



Average Metric: 47.472619047619055 / 94  (50.5):  93%|▉| 93/100 [00:49<00:06,  1.15i
Average Metric: 47.472619047619055 / 94  (50.5):  94%|▉| 94/100 [00:49<00:03,  1.53i
Average Metric: 47.472619047619055 / 95  (50.0):  94%|▉| 94/100 [00:49<00:03,  1.53i
Average Metric: 47.472619047619055 / 95  (50.0):  95%|▉| 95/100 [00:49<00:03,  1.62i
Average Metric: 48.472619047619055 / 96  (50.5):  95%|▉| 95/100 [00:51<00:03,  1.62i
Average Metric: 48.472619047619055 / 96  (50.5):  96%|▉| 96/100 [00:51<00:03,  1.04i
Average Metric: 48.472619047619055 / 97  (50.0):  96%|▉| 96/100 [00:51<00:03,  1.04i
Average Metric: 48.472619047619055 / 97  (50.0):  97%|▉| 97/100 [00:51<00:02,  1.31i
Average Metric: 48.80595238095239 / 98  (49.8):  97%|▉| 97/100 [00:57<00:02,  1.31it
Average Metric: 48.80595238095239 / 98  (49.8):  98%|▉| 98/100 [00:57<00:04,  2.24s/
Average Metric: 48.80595238095239 / 99  (49.3):  98%|▉| 98/100 [01:00<00:04,  2.24s/
Average Metric: 48.80595238095239 / 99  (49.3):  99%|▉| 99/100 [

Average Metric: 49.80595238095239 / 100  (49.8%)



/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)
/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:263: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.7142857142857143' '1.0' '0.3333333333333333' '0.0'
 '0.3333333333333333']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,review,example_aspects_with_label,aspects_list,pred_aspects_with_label,validate_venn
0,""" The menu includes pub fare--burgers, steaks and shepherds pie--and even a portabella lasagna for those black sheep known as ""vegetarians.","{'menu': 0, 'burgers': 0, 'steaks': 0, 'shepherds pie': 0, 'portabella lasagna': 0, 'pub fare': 0}","dict_keys(['menu', 'burgers', 'steaks', 'shepherds pie', 'portabella lasagna', 'pub fare'])","{'pub fare': 0, 'burgers': 0, 'steaks': 0, 'shepherds pie': 0, 'portabella lasagna': 0, 'vegetarians': 0}",0.7142857142857143
1,(The sashimi is cut a little thinly.,{'sashimi': -1},dict_keys(['sashimi']),{'sashimi': -1},1.0
2,") It's not the best Japanese restaurant in the East Village, but it's a pretty solid one for its modest prices, and worth repeat visits.",{'prices': 1},dict_keys(['prices']),"{'Japanese restaurant': 0, 'East Village': 0, 'prices': 1}",0.3333333333333333
3,15% gratuity automatically added to the bill.,"{'bill': -1, 'gratuity': -1}","dict_keys(['bill', 'gratuity'])",{'gratuity': 0},0.0
4,"A beautiful atmosphere, perfect for drinks and/or appetizers.","{'atmosphere': 1, 'drinks': 0, 'appetizers': 0}","dict_keys(['atmosphere', 'drinks', 'appetizers'])","{'atmosphere': 1, 'drinks': 1, 'appetizers': 1}",0.3333333333333333


In [159]:
uncompiled_pipeline_score, compiled_pipeline_score

(41.21, 49.81)

## V2

In [161]:
class Review2Aspects(dspy.Signature):
    """
    Identify aspects from a customer review. The aspects must be words or phrases in the review. The aspect
    is typically a noun, and it must be the narrowest word or phrase possible. The response must be a valid Python list,
    where each item is an aspect.
    """

    review: str = dspy.InputField(desc="a customer review")
    aspects_list: str = dspy.OutputField(desc=dedent("""
        A list of all the aspects in the customer review. Must be a valid Python list.
        """))

In [162]:
class Aspects2Reasons(dspy.Signature):
    """
    Using the customer review and list of aspects, reason about whether each aspect is positive, neutral, or negative.
    Think deeply about each aspect in the context of the entire review.
    """

    review: str = dspy.InputField(desc="a customer review")
    aspects_list: str = dspy.InputField(desc=dedent("""
        A list of all the aspects in the customer review
        """))
    reasons: str = dspy.OutputField(desc=dedent("""
        A list of all the aspects in the customer review. Under each aspect, there are some thoughts or reasons
        about whether the aspect is positive, neutral, or negative.
        """))

In [163]:
class Reasons2Labels(dspy.Signature):
    """
    Labels each aspect as 1 meaning positive, 0 meaning neutral, and -1 meaning negative. The response must be a 
    valid Python dictionary, where each key is an aspect and the value is the integer label.
    """

    aspects_list: str = dspy.InputField(desc=dedent("""
        A list of all the aspects in the customer review
        """))
    reasons: str = dspy.InputField(desc=dedent("""
        A list of all the aspects in the customer review. Under each aspect, there are some thoughts or reasons
        about whether the aspect is positive, neutral, or negative.
        """))
    aspects_with_label: dict = dspy.OutputField(format=dict, desc=dedent("""
        a single Python dictionary, where each key is an aspect and the value is the label,
        with label 1 indicating positive sentiment, 0 indicating neutral sentiment, and -1
        indicating negative sentiment.
        """))

In [164]:
class ABSAPipelineV2(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_aspects = dspy.TypedChainOfThought(Review2Aspects)
        self.generate_reasons = dspy.ChainOfThought(Aspects2Reasons)
        self.generate_labels = dspy.TypedChainOfThought(Reasons2Labels)

    def forward(self, review):
        aspects_list = self.generate_aspects(review=review)
        reasons = self.generate_reasons(review=review, aspects_list=aspects_list.aspects_list)
        pred = self.generate_labels(aspects_list=aspects_list.aspects_list, reasons=reasons.reasons)
        return dspy.Prediction(aspects_with_label=pred.aspects_with_label)

In [165]:
uncompiled_pipeline = ABSAPipelineV2()

In [166]:
pred = uncompiled_pipeline(dspy_test[0].review)

In [167]:
pred

Prediction(
    aspects_with_label={'burgers': 1, 'steaks': 1, 'shepherds pie': 1, 'portabella lasagna': 1, 'vegetarians': 1}
)

In [173]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

optimizer = BootstrapFewShotWithRandomSearch(metric=validate_venn, metric_threshold=0.8)
compiled_pipeline = optimizer.compile(ABSAPipelineV2(), teacher=ABSAPipelineV2(), trainset=dspy_train[:30])

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 16 candidate sets.




  0%|                                                        | 0/30 [00:00<?, ?it/s]

Average Metric: 0.0 / 1  (0.0):   0%|                        | 0/30 [00:00<?, ?it/s]

Average Metric: 0.0 / 1  (0.0):   3%|▌               | 1/30 [00:00<00:03,  8.86it/s]

Average Metric: 0.0 / 2  (0.0):   3%|▌               | 1/30 [00:00<00:03,  8.86it/s]

Average Metric: 0.3333333333333333 / 3  (11.1):   7%| | 2/30 [00:00<00:03,  8.86it/s

Average Metric: 0.3333333333333333 / 4  (8.3):  10%| | 3/30 [00:00<00:03,  8.86it/s]

Average Metric: 0.3333333333333333 / 5  (6.7):  13%|▏| 4/30 [00:00<00:02,  8.86it/s]

Average Metric: 0.3333333333333333 / 6  (5.6):  17%|▏| 5/30 [00:00<00:02,  8.86it/s]

Average Metric: 0.3333333333333333 / 6  (5.6):  20%|▏| 6/30 [00:00<00:00, 31.76it/s]

Average Metric: 0.8333333333333333 / 7  (11.9):  20%|▏| 6/30 [00:00<00:00, 31.76it/s

Average Metric: 1.3333333333333333 / 8  (16.7):  23%|▏| 7/30 [00:00<00:00, 31.76it/s

Average Metric: 1.3333333333333333 / 9  (14.8):  27%

Average Metric: 8.38888888888889 / 30  (28.0%)
Score: 27.96 for set: [0, 0, 0]
New best score: 27.96 for seed -3
Scores so far: [27.96]
Best score: 27.96




  0%|                                                        | 0/30 [00:00<?, ?it/s]

Average Metric: 0.3333333333333333 / 1  (33.3):   0%|        | 0/30 [00:00<?, ?it/s]

Average Metric: 0.3333333333333333 / 2  (16.7):   3%| | 1/30 [00:00<00:02, 11.76it/s

Average Metric: 1.3333333333333333 / 3  (44.4):   7%| | 2/30 [00:00<00:01, 19.29it/s

Average Metric: 1.3333333333333333 / 3  (44.4):  10%| | 3/30 [00:00<00:01, 26.35it/s

Average Metric: 2.333333333333333 / 4  (58.3):  10%| | 3/30 [00:00<00:01, 26.35it/s]

Average Metric: 2.833333333333333 / 5  (56.7):  13%|▏| 4/30 [00:00<00:00, 26.35it/s]

Average Metric: 3.233333333333333 / 6  (53.9):  17%|▏| 5/30 [00:00<00:00, 26.35it/s]

Average Metric: 4.2333333333333325 / 7  (60.5):  20%|▏| 6/30 [00:00<00:00, 26.35it/s

Average Metric: 4.2333333333333325 / 7  (60.5):  23%|▏| 7/30 [00:00<00:00, 33.02it/s

Average Metric: 4.333333333333332 / 8  (54.2):  23%|▏| 7/30 [00:00<00:00, 33.02it/s]

Average Metric: 4.933333333333332 / 9  (54.8):  27%|

Average Metric: 15.4 / 30  (51.3%)
Score: 51.33 for set: [16, 16, 16]
New best score: 51.33 for seed -2
Scores so far: [27.96, 51.33]
Best score: 51.33


...

Average Metric: 12.783333333333333 / 30  (42.6%)
Score: 42.61 for set: [16, 16, 16]
Scores so far: [27.96, 51.33, 45.78, 48.22, 47.11, 49.83, 44.36, 54.28, 55.73, 51.97, 50.83, 50.92, 62.61, 52.11, 52.44, 48.78, 40.39, 42.61]
Best score: 62.61
Average of max per entry across top 1 scores: 0.6261111111111111
Average of max per entry across top 2 scores: 0.7694444444444445
Average of max per entry across top 3 scores: 0.8544444444444445
Average of max per entry across top 5 scores: 0.9183333333333333
Average of max per entry across top 8 scores: 0.9441666666666666
Average of max per entry across top 9999 scores: 0.9508333333333334




  0%|                                                        | 0/30 [00:00<?, ?it/s]

 33%|███████████████▋                               | 10/30 [00:00<00:00, 50.02it/s]


Bootstrapped 2 full traces after 11 examples in round 0.




  0%|                                                        | 0/30 [00:00<?, ?it/s]

Average Metric: 1.0 / 1  (100.0):   0%|                      | 0/30 [00:00<?, ?it/s]

Average Metric: 1.2857142857142856 / 2  (64.3):   3%| | 1/30 [00:00<00:02, 10.73it/s

Average Metric: 1.2857142857142856 / 2  (64.3):   7%| | 2/30 [00:00<00:01, 19.10it/s

Average Metric: 2.2857142857142856 / 3  (76.2):   7%| | 2/30 [00:00<00:01, 19.10it/s

Average Metric: 2.2857142857142856 / 4  (57.1):  10%| | 3/30 [00:00<00:01, 19.10it/s

Average Metric: 3.2857142857142856 / 5  (65.7):  13%|▏| 4/30 [00:00<00:01, 19.10it/s

Average Metric: 3.619047619047619 / 6  (60.3):  17%|▏| 5/30 [00:00<00:01, 19.10it/s]

Average Metric: 3.619047619047619 / 7  (51.7):  20%|▏| 6/30 [00:00<00:01, 19.10it/s]

Average Metric: 3.619047619047619 / 7  (51.7):  23%|▏| 7/30 [00:00<00:00, 33.29it/s]

Average Metric: 3.9523809523809526 / 8  (49.4):  23%|▏| 7/30 [00:00<00:00, 33.29it/s

Average Metric: 4.152380952380953 / 9  (46.1):  27%|

Average Metric: 13.735714285714286 / 30  (45.8%)
Score: 45.79 for set: [16, 16, 16]
Scores so far: [27.96, 51.33, 45.78, 48.22, 47.11, 49.83, 44.36, 54.28, 55.73, 51.97, 50.83, 50.92, 62.61, 52.11, 52.44, 48.78, 40.39, 42.61, 45.79]
Best score: 62.61
Average of max per entry across top 1 scores: 0.6261111111111111
Average of max per entry across top 2 scores: 0.7694444444444445
Average of max per entry across top 3 scores: 0.8544444444444445
Average of max per entry across top 5 scores: 0.9183333333333333
Average of max per entry across top 8 scores: 0.9441666666666666
Average of max per entry across top 9999 scores: 0.9522222222222222
19 candidate programs found.


In [175]:
# compiled_pipeline = ABSAPipelineV2()
# ABSAPipelineV2.load(compiled_pipeline, "./ABSAPipelineV2.json")
evaluate_test(compiled_pipeline, metric=validate_f1)



  0%|                                                       | 0/100 [00:00<?, ?it/s]

Average Metric: 0.4 / 1  (40.0):   0%|                      | 0/100 [00:00<?, ?it/s]

Average Metric: 0.4 / 1  (40.0):   1%|▏             | 1/100 [00:00<00:15,  6.39it/s]

Average Metric: 0.4 / 2  (20.0):   1%|▏             | 1/100 [00:00<00:15,  6.39it/s]

Average Metric: 0.9714285714285714 / 3  (32.4):   2%| | 2/100 [00:00<00:15,  6.39it/

Average Metric: 1.3047619047619048 / 4  (32.6):   3%| | 3/100 [00:00<00:15,  6.39it/

Average Metric: 1.8047619047619048 / 5  (36.1):   4%| | 4/100 [00:00<00:15,  6.39it/

Average Metric: 2.3047619047619046 / 6  (38.4):   5%| | 5/100 [00:00<00:14,  6.39it/

Average Metric: 2.3047619047619046 / 6  (38.4):   6%| | 6/100 [00:00<00:03, 26.68it/

Average Metric: 3.138095238095238 / 7  (44.8):   6%| | 6/100 [00:00<00:03, 26.68it/s

Average Metric: 4.138095238095238 / 8  (51.7):   7%| | 7/100 [00:00<00:03, 26.68it/s

Average Metric: 4.138095238095238 / 9  (46.0):   8%|

Average Metric: 54.79300144300142 / 100  (54.8%)



/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:263: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.8333333333333334' '0.0' '0.4' '0.0' '0.5']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,review,example_aspects_with_label,aspects_list,pred_aspects_with_label,validate_f1
0,""" The menu includes pub fare--burgers, steaks and shepherds pie--and even a portabella lasagna for those black sheep known as ""vegetarians.","{'menu': 0, 'burgers': 0, 'steaks': 0, 'shepherds pie': 0, 'portabella lasagna': 0, 'pub fare': 0}","dict_keys(['menu', 'burgers', 'steaks', 'shepherds pie', 'portabella lasagna', 'pub fare'])","{'menu': 0, 'pub fare': 0, 'burgers': 0, 'steaks': 0, 'shepherds pie': 0, 'portabella lasagna': 1, 'vegetarians': 1}",0.8333333333333334
1,(The sashimi is cut a little thinly.,{'sashimi': -1},dict_keys(['sashimi']),{'sashimi': 0},0.0
2,") It's not the best Japanese restaurant in the East Village, but it's a pretty solid one for its modest prices, and worth repeat visits.",{'prices': 1},dict_keys(['prices']),"{'Japanese restaurant': 0, 'East Village': 0, 'prices': 1, 'repeat visits': 1}",0.4
3,15% gratuity automatically added to the bill.,"{'bill': -1, 'gratuity': -1}","dict_keys(['bill', 'gratuity'])","{'gratuity': 0, 'bill': 0}",0.0
4,"A beautiful atmosphere, perfect for drinks and/or appetizers.","{'atmosphere': 1, 'drinks': 0, 'appetizers': 0}","dict_keys(['atmosphere', 'drinks', 'appetizers'])","{'atmosphere': 1, 'drinks': 1, 'appetizers': 1}",0.5


54.79

In [174]:
compiled_pipeline.save("./ABSAPipelineV2thresh.json")

In [123]:
# compiled_pipeline.save("./ABSAPipelineV2.json")
compiled_v2 = ABSAPipelineV2()
ABSAPipelineV2.load(compiled_v2, "./ABSAPipelineV2.json")

In [489]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

optimizer = BootstrapFewShotWithRandomSearch(metric=validate_f1)
compiled_f1 = optimizer.compile(ABSAPipelineV2(), teacher=ABSAPipelineV2(), trainset=dspy_train[:30])

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 16 candidate sets.


Average Metric: 8.623809523809523 / 30  (28.7): 100%|█| 30/30 [00:00<00:00, 52.17it/
/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


Average Metric: 8.623809523809523 / 30  (28.7%)
Score: 28.75 for set: [0, 0, 0]
New best score: 28.75 for seed -3
Scores so far: [28.75]
Best score: 28.75


Average Metric: 11.357142857142858 / 30  (37.9): 100%|█| 30/30 [00:00<00:00, 54.51it


Average Metric: 11.357142857142858 / 30  (37.9%)
Score: 37.86 for set: [16, 16, 16]
New best score: 37.86 for seed -2
Scores so far: [28.75, 37.86]
Best score: 37.86


...

Average Metric: 10.933333333333334 / 30  (36.4%)
Score: 36.44 for set: [16, 16, 16]
Scores so far: [28.75, 37.86, 36.97, 35.27, 31.63, 39.97, 29.11, 39.44, 37.78, 42.22, 34.05, 40.63, 39.22, 33.97, 36.52, 38.57, 34.44, 36.44]
Best score: 42.22
Average of max per entry across top 1 scores: 0.4222222222222222
Average of max per entry across top 2 scores: 0.631111111111111
Average of max per entry across top 3 scores: 0.6974603174603174
Average of max per entry across top 5 scores: 0.883015873015873
Average of max per entry across top 8 scores: 0.9396825396825397
Average of max per entry across top 9999 scores: 1.0


 17%|████████                                        | 5/30 [00:00<00:00, 49.38it/s]


Bootstrapped 2 full traces after 6 examples in round 0.


Average Metric: 10.757142857142856 / 30  (35.9): 100%|█| 30/30 [00:00<00:00, 48.88it

Average Metric: 10.757142857142856 / 30  (35.9%)
Score: 35.86 for set: [16, 16, 16]
Scores so far: [28.75, 37.86, 36.97, 35.27, 31.63, 39.97, 29.11, 39.44, 37.78, 42.22, 34.05, 40.63, 39.22, 33.97, 36.52, 38.57, 34.44, 36.44, 35.86]
Best score: 42.22
Average of max per entry across top 1 scores: 0.4222222222222222
Average of max per entry across top 2 scores: 0.631111111111111
Average of max per entry across top 3 scores: 0.6974603174603174
Average of max per entry across top 5 scores: 0.883015873015873
Average of max per entry across top 8 scores: 0.9396825396825397
Average of max per entry across top 9999 scores: 1.0
19 candidate programs found.


In [500]:
compiled_f1_score = evaluate_test(compiled_f1, metric=validate_f1)
compiled_f1_score



  0%|                                                       | 0/100 [00:00<?, ?it/s]

Average Metric: 0 / 1  (0.0):   0%|                         | 0/100 [00:00<?, ?it/s]

Average Metric: 0 / 1  (0.0):   1%|▏                | 1/100 [00:00<00:21,  4.57it/s]

Average Metric: 0 / 2  (0.0):   1%|▏                | 1/100 [00:00<00:21,  4.57it/s]

Average Metric: 0.0 / 3  (0.0):   2%|▎              | 2/100 [00:00<00:21,  4.57it/s]

Average Metric: 0.5 / 4  (12.5):   3%|▍             | 3/100 [00:00<00:21,  4.57it/s]

Average Metric: 1.0 / 5  (20.0):   4%|▌             | 4/100 [00:00<00:20,  4.57it/s]

Average Metric: 1.0 / 5  (20.0):   5%|▋             | 5/100 [00:00<00:05, 18.24it/s]

Average Metric: 1.0 / 6  (16.7):   5%|▋             | 5/100 [00:00<00:05, 18.24it/s]

Average Metric: 2.0 / 7  (28.6):   6%|▊             | 6/100 [00:00<00:05, 18.24it/s]

Average Metric: 3.0 / 8  (37.5):   7%|▉             | 7/100 [00:00<00:05, 18.24it/s]

Average Metric: 3.0 / 8  (37.5):   8%|█             

Error for example in dev set: 		 ('Too many retries trying to get the correct output format. Try simplifying the requirements.', {'aspects_with_labels': "ValueError('json output should start and end with { and }')"})


Average Metric: 31.519047619047623 / 72  (43.8):  71%|▋| 71/100 [00:03<00:01, 14.60i

Average Metric: 32.18571428571429 / 73  (44.1):  72%|▋| 72/100 [00:03<00:01, 14.60it

Average Metric: 33.18571428571429 / 74  (44.8):  73%|▋| 73/100 [00:03<00:01, 14.60it

Average Metric: 33.18571428571429 / 75  (44.2):  74%|▋| 74/100 [00:03<00:01, 14.60it

Average Metric: 33.18571428571429 / 76  (43.7):  75%|▊| 75/100 [00:03<00:01, 14.60it

Average Metric: 33.18571428571429 / 76  (43.7):  76%|▊| 76/100 [00:04<00:01, 19.24it

Average Metric: 33.58571428571429 / 77  (43.6):  76%|▊| 76/100 [00:04<00:01, 19.24it

Average Metric: 33.58571428571429 / 78  (43.1):  77%|▊| 77/100 [00:04<00:01, 19.24it

Average Metric: 33.58571428571429 / 79  (42.5):  78%|▊| 78/100 [00:04<00:01, 19.24it

Average Metric: 33.58571428571429 / 80  (42.0):  79%|▊| 79/100 [00:04<00:01, 19.24it

Average Metric: 33.58571428571429 / 81  (41.5):  80%|▊| 80/100 [00:04<00:01, 19.24it

Average Metric: 33.58571428571429 / 81  (41.5):  81%|▊

Error for example in dev set: 		 ('Too many retries trying to get the correct output format. Try simplifying the requirements.', {'aspects_with_labels': 'Field required: value (error type: missing)'})
Average Metric: 41.41904761904762 / 100  (41.4%)



/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)
/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:263: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.0' '0.0' '0.0' '0.0' '1.0']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,review,example_aspects,pred_aspects,validate_f1,aspects
0,""" The menu includes pub fare--burgers, steaks and shepherds pie--and even a portabella lasagna for those black sheep known as ""vegetarians.","{'menu': 0, 'burgers': 0, 'steaks': 0, 'shepherds pie': 0, 'portabella lasagna': 0, 'pub fare': 0}","{'menu': 0, 'pub fare': 0, 'burgers': 0, 'steaks': 0, 'shepherds pie': 0, 'portabella lasagna': 1, 'vegetarians': 1}",0.0,nan
1,(The sashimi is cut a little thinly.,{'sashimi': -1},{'sashimi': 0},0.0,nan
2,") It's not the best Japanese restaurant in the East Village, but it's a pretty solid one for its modest prices, and worth repeat visits.",{'prices': 1},"{'Japanese restaurant': 0, 'East Village': 0, 'modest prices': 1, 'repeat visits': 1}",0.0,nan
3,15% gratuity automatically added to the bill.,"{'bill': -1, 'gratuity': -1}",{'gratuity': 0},0.0,nan
4,"A beautiful atmosphere, perfect for drinks and/or appetizers.","{'atmosphere': 1, 'drinks': 0, 'appetizers': 0}","{'atmosphere': 1, 'drinks': 0, 'appetizers': 0}",1.0,nan


41.42

## With Optuna

In [189]:
compiled_v2 = ABSAPipelineV2()
ABSAPipelineV2.load(compiled_v2, "./ABSAPipelineV2.json")

from dspy.teleprompt import BootstrapFewShotWithOptuna

optimizer = BootstrapFewShotWithOptuna(metric=validate_venn)
compiled_optuna = optimizer.compile(ABSAPipelineV2(), teacher=compiled_v2, trainset=dspy_train[30:60], max_demos=10)

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 16 candidate sets.



100%|█████████████████████████████████████████| 30/30 [02:52<00:00,  5.74s/it]
[I 2024-04-30 08:59:58,267] A new study created in memory with name: no-name-39f1490d-625f-4ff4-98bf-48dc7be0698c


Bootstrapped 6 full traces after 30 examples in round 0.



Average Metric: 1.1 / 5  (22.0):  17%|█▌       | 5/30 [00:08<00:31,  1.24s/it]
Average Metric: 1.2428571428571429 / 6  (20.7):  17%|▏| 5/30 [00:10<00:31,  1.
Average Metric: 1.2428571428571429 / 6  (20.7):  20%|▏| 6/30 [00:10<00:39,  1.
Average Metric: 1.7428571428571429 / 7  (24.9):  20%|▏| 6/30 [00:11<00:39,  1.
Average Metric: 1.7428571428571429 / 7  (24.9):  23%|▏| 7/30 [00:11<00:27,  1.
Average Metric: 2.0761904761904764 / 8  (26.0):  23%|▏| 7/30 [00:11<00:27,  1.
Average Metric: 2.0761904761904764 / 8  (26.0):  27%|▎| 8/30 [00:11<00:18,  1.
Average Metric: 2.4761904761904763 / 9  (27.5):  27%|▎| 8/30 [00:11<00:18,  1.
Average Metric: 2.4761904761904763 / 9  (27.5):  30%|▎| 9/30 [00:11<00:15,  1.
Average Metric: 2.9761904761904763 / 10  (29.8):  30%|▎| 9/30 [00:12<00:15,  1
Average Metric: 2.9761904761904763 / 10  (29.8):  33%|▎| 10/30 [00:12<00:17,  
Average Metric: 2.9761904761904763 / 11  (27.1):  33%|▎| 10/30 [00:13<00:17,  
Average Metric: 2.9761904761904763 / 11  (27.1):  3

Average Metric: 11.642857142857144 / 30  (38.8%)


...

Average Metric: 12.516666666666667 / 30  (41.7%)



Average Metric: 1.9 / 6  (31.7):  20%|█▊       | 6/30 [00:07<00:25,  1.06s/it]
Average Metric: 2.5666666666666664 / 7  (36.7):  20%|▏| 6/30 [00:07<00:25,  1.
Average Metric: 2.5666666666666664 / 7  (36.7):  23%|▏| 7/30 [00:07<00:18,  1.
Average Metric: 2.5666666666666664 / 8  (32.1):  23%|▏| 7/30 [00:08<00:18,  1.
Average Metric: 2.5666666666666664 / 8  (32.1):  27%|▎| 8/30 [00:08<00:13,  1.
Average Metric: 3.4 / 10  (34.0):  33%|██▎    | 10/30 [00:10<00:20,  1.02s/it]
Average Metric: 3.7333333333333334 / 11  (33.9):  33%|▎| 10/30 [00:10<00:20,  
Average Metric: 3.7333333333333334 / 11  (33.9):  37%|▎| 11/30 [00:10<00:14,  
Average Metric: 4.233333333333333 / 12  (35.3):  37%|▎| 11/30 [00:11<00:14,  1
Average Metric: 4.233333333333333 / 12  (35.3):  40%|▍| 12/30 [00:11<00:10,  1
Average Metric: 4.233333333333333 / 13  (32.6):  40%|▍| 12/30 [00:12<00:10,  1
Average Metric: 4.233333333333333 / 13  (32.6):  43%|▍| 13/30 [00:12<00:13,  1
Average Metric: 4.233333333333333 / 14  (30.2):  43

Average Metric: 13.300000000000002 / 30  (44.3%)



Average Metric: 1.0 / 1  (100.0):   3%|▎       | 1/30 [00:03<01:38,  3.40s/it]
Average Metric: 1.6666666666666665 / 2  (83.3):   3%| | 1/30 [00:03<01:38,  3.
Average Metric: 2.1666666666666665 / 3  (72.2):   7%| | 2/30 [00:04<01:35,  3.
Average Metric: 2.1666666666666665 / 3  (72.2):  10%| | 3/30 [00:04<00:31,  1.
Average Metric: 2.6666666666666665 / 4  (66.7):  10%| | 3/30 [00:04<00:31,  1.
Average Metric: 2.9166666666666665 / 5  (58.3):  13%|▏| 4/30 [00:05<00:29,  1.
Average Metric: 2.9166666666666665 / 5  (58.3):  17%|▏| 5/30 [00:05<00:22,  1.
Average Metric: 3.4166666666666665 / 6  (56.9):  17%|▏| 5/30 [00:05<00:22,  1.
Average Metric: 3.4166666666666665 / 7  (48.8):  20%|▏| 6/30 [00:05<00:21,  1.
Average Metric: 3.4166666666666665 / 7  (48.8):  23%|▏| 7/30 [00:05<00:12,  1.
Average Metric: 3.8166666666666664 / 8  (47.7):  23%|▏| 7/30 [00:05<00:12,  1.
Average Metric: 3.8166666666666664 / 8  (47.7):  27%|▎| 8/30 [00:05<00:09,  2.
Average Metric: 3.8166666666666664 / 9  (42.4):  27

Average Metric: 14.383333333333335 / 30  (47.9%)
Best score: 49.44
Best program: generate_aspects.predictor = Predict(StringSignature(review -> reasoning, aspects_list
    instructions='\n    Identify aspects from a customer review. The aspects must be words or phrases in the review. The aspect\n    is typically a noun, and it must be the narrowest word or phrase possible. The response must be a valid Python list,\n    where each item is an aspect.\n    '
    review = Field(annotation=str required=True json_schema_extra={'desc': 'a customer review', '__dspy_field_type': 'input', 'prefix': 'Review:'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${produce the aspects_list}. We ...', '__dspy_field_type': 'output'})
    aspects_list = Field(annotation=str required=True json_schema_extra={'desc': '\nA list of all the aspects in the customer review. Must be a valid Python list.\n', '__dspy_field_t

In [190]:
evaluate_test(compiled_optuna, metric=validate_f1)


  0%|                                                 | 0/100 [00:00<?, ?it/s]

Backing off 0.3 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 0.8 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 0.3 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 0.9 seconds after 2 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 1.3 seconds after 2 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 1.4 seconds after 2 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}



Average Metric: 1.4 / 3  (46.7):   3%|▏       | 3/100 [00:06<03:01,  1.87s/it]

Backing off 0.5 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 2.9 seconds after 3 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 3.4 seconds after 3 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}



Average Metric: 1.4 / 4  (35.0):   4%|▎       | 4/100 [00:08<02:52,  1.80s/it]
Average Metric: 2.0666666666666664 / 5  (41.3):   4%| | 4/100 [00:08<02:52,  1
Average Metric: 2.0666666666666664 / 5  (41.3):   5%| | 5/100 [00:08<02:07,  1

...


Average Metric: 45.712698412698394 / 92  (49.7):  91%|▉| 91/100 [01:46<00:08, 
Average Metric: 45.712698412698394 / 92  (49.7):  92%|▉| 92/100 [01:46<00:07, 
Average Metric: 45.712698412698394 / 93  (49.2):  92%|▉| 92/100 [01:46<00:07, 
Average Metric: 45.712698412698394 / 93  (49.2):  93%|▉| 93/100 [01:46<00:06, 
Average Metric: 45.712698412698394 / 94  (48.6):  93%|▉| 93/100 [01:47<00:06, 
Average Metric: 45.712698412698394 / 94  (48.6):  94%|▉| 94/100 [01:47<00:04, 
Average Metric: 45.712698412698394 / 95  (48.1):  94%|▉| 94/100 [01:47<00:04, 
Average Metric: 45.712698412698394 / 95  (48.1):  95%|▉| 95/100 [01:47<00:03, 
Average Metric: 45.712698412698394 / 96  (47.6):  95%|▉| 95/100 [01:48<00:03, 
Average Metric: 45.712698412698394 / 96  (47.6):  96%|▉| 96/100 [01:48<00:02, 
Average Metric: 45.712698412698394 / 97  (47.1):  96%|▉| 96/100 [01:49<00:02, 
Average Metric: 45.712698412698394 / 97  (47.1):  97%|▉| 97/100 [01:49<00:02, 
Average Metric: 46.04603174603173 / 98  (47.0):  97

Average Metric: 47.212698412698394 / 100  (47.2%)



/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:263: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.4' '0.0' '0.0' '0.6666666666666666' '0.4']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,review,example_aspects_with_label,aspects_list,pred_aspects_with_label,validate_f1
0,""" The menu includes pub fare--burgers, steaks and shepherds pie--and even a portabella lasagna for those black sheep known as ""vegetarians.","{'menu': 0, 'burgers': 0, 'steaks': 0, 'shepherds pie': 0, 'portabella lasagna': 0, 'pub fare': 0}","dict_keys(['menu', 'burgers', 'steaks', 'shepherds pie', 'portabella lasagna', 'pub fare'])","{'burgers': 0, 'steaks': 0, ""shepherd's pie"": 0, 'portabella lasagna': 1, 'vegetarian options': 1}",0.4
1,(The sashimi is cut a little thinly.,{'sashimi': -1},dict_keys(['sashimi']),{'thickness': -1},0.0
2,") It's not the best Japanese restaurant in the East Village, but it's a pretty solid one for its modest prices, and worth repeat visits.",{'prices': 1},dict_keys(['prices']),"{'Japanese restaurant': 0, 'East Village': 0, 'quality': 1, 'modest prices': 1, 'repeat visits': 1}",0.0
3,15% gratuity automatically added to the bill.,"{'bill': -1, 'gratuity': -1}","dict_keys(['bill', 'gratuity'])",{'gratuity': -1},0.6666666666666666
4,"A beautiful atmosphere, perfect for drinks and/or appetizers.","{'atmosphere': 1, 'drinks': 0, 'appetizers': 0}","dict_keys(['atmosphere', 'drinks', 'appetizers'])","{'atmosphere': 1, 'drinks/appetizers': 1}",0.4


47.21

In [188]:
compiled_optuna.save("./ABSAPipelineV2Optuna.json")

## MIPRO

In [125]:
from dspy.teleprompt import MIPRO

optimizer = MIPRO(metric=validate_f1)
kwargs = dict(num_threads=10, display_progress=True, display_table=5)
compiled_mipro = optimizer.compile(ABSAPipelineV2(), trainset=dspy_train[:30], num_trials=50, max_bootstrapped_demos=3, max_labeled_demos=5, eval_kwargs=kwargs)


Please be advised that based on the parameters you have set, the maximum number of LM calls is projected as follows:

- Task Model: 30 examples in dev set * 50 trials * # of LM calls in your program = (1500 * # of LM calls in your program) task model calls
- Prompt Model: # data summarizer calls (max 10) + 10 * 3 lm calls in program = 40 prompt model calls

Estimated Cost Calculation:

Total Cost = (Number of calls to task model * (Avg Input Token Length per Call * Task Model Price per Input Token + Avg Output Token Length per Call * Task Model Price per Output Token) 
            + (Number of calls to prompt model * (Avg Input Token Length per Call * Task Prompt Price per Input Token + Avg Output Token Length per Call * Prompt Model Price per Output Token).

For a preliminary estimate of potential costs, we recommend you perform your own calculations based on the task
and prompt models you intend to use. If the projected costs exceed your budget or expectations, you may consider:

- 

Do you wish to continue? (y/n):  y



 40%|██████████████████▊                            | 12/30 [00:00<00:00, 45.57it/s]


Bootstrapped 3 full traces after 13 examples in round 0.



 57%|██████████████████████████▋                    | 17/30 [00:00<00:00, 54.59it/s]


...

Error for example in dev set: 		 'NoneType' object is not callable



Average Metric: 0.0 / 100  (0.0): 100%|███████████| 100/100 [03:58<00:00,  2.38s/it]

Error for example in dev set: 		 'NoneType' object is not callable
Average Metric: 0.0 / 100  (0.0%)



/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


AssertionError: 

In [131]:
ABSAPipelineV2.load(compiled_mipro, "./ABSAPipelineV2MIPRO.json")

In [132]:
eval_score = evaluate_test(compiled_mipro, metric=validate_f1)


Average Metric: 3.15 / 4  (78.8):   3%|▍            | 3/100 [00:00<00:15,  6.30it/s]
Average Metric: 3.877272727272727 / 5  (77.5):   4%| | 4/100 [00:00<00:15,  6.30it/s
Average Metric: 4.099494949494949 / 6  (68.3):   5%| | 5/100 [00:00<00:15,  6.30it/s
Average Metric: 5.099494949494949 / 7  (72.8):   6%| | 6/100 [00:00<00:14,  6.30it/s
Average Metric: 5.099494949494949 / 7  (72.8):   7%| | 7/100 [00:00<00:03, 27.78it/s
Average Metric: 5.432828282828282 / 8  (67.9):   7%| | 7/100 [00:00<00:03, 27.78it/s
Average Metric: 5.432828282828282 / 9  (60.4):   8%| | 8/100 [00:00<00:03, 27.78it/s
Average Metric: 5.432828282828282 / 10  (54.3):   9%| | 9/100 [00:00<00:03, 27.78it/
Average Metric: 6.099494949494949 / 11  (55.4):  10%| | 10/100 [00:00<00:03, 27.78it
Average Metric: 6.899494949494949 / 12  (57.5):  11%| | 11/100 [00:00<00:03, 27.78it
Average Metric: 7.399494949494949 / 13  (56.9):  12%| | 12/100 [00:00<00:03, 27.78it
Average Metric: 8.066161616161615 / 14  (57.6):  13%|▏| 13/100 [

Average Metric: 58.43549783549781 / 100  (58.4%)


,review,example_aspects_with_label,aspects_list,pred_aspects_with_label,validate_f1
0,""" The menu includes pub fare--burgers, steaks and shepherds pie--and even a portabella lasagna for those black sheep known as ""vegetarians.","{'menu': 0, 'burgers': 0, 'steaks': 0, 'shepherds pie': 0, 'portabella lasagna': 0, 'pub fare': 0}","dict_keys(['menu', 'burgers', 'steaks', 'shepherds pie', 'portabella lasagna', 'pub fare'])","{'menu': 1, 'pub fare': 0, 'burgers': 0, 'steaks': 0, 'shepherds pie': 0, 'portabella lasagna': 1, 'vegetarians': 1}",0.7272727272727273
1,(The sashimi is cut a little thinly.,{'sashimi': -1},dict_keys(['sashimi']),{'sashimi': -1},1.0
2,") It's not the best Japanese restaurant in the East Village, but it's a pretty solid one for its modest prices, and worth repeat visits.",{'prices': 1},dict_keys(['prices']),"{'Japanese restaurant': 0, 'East Village': 0, 'prices': 1, 'repeat visits': 1}",0.4
3,15% gratuity automatically added to the bill.,"{'bill': -1, 'gratuity': -1}","dict_keys(['bill', 'gratuity'])","{'gratuity': 0, 'bill': 0}",0.0
4,"A beautiful atmosphere, perfect for drinks and/or appetizers.","{'atmosphere': 1, 'drinks': 0, 'appetizers': 0}","dict_keys(['atmosphere', 'drinks', 'appetizers'])","{'atmosphere': 1, 'drinks': 0, 'appetizers': 0}",1.0


In [133]:
eval_score, eval_score_f1

(58.44, 56.71)

## V3

In [77]:
class Review2AspectsList(dspy.Signature):
    """
    Identify aspects from a customer review. The aspects must be words or phrases in the review. The aspect
    is typically a noun, and it must be the narrowest word or phrase possible. The response must be a valid Python list.
    """

    review: str = dspy.InputField(desc="a customer review")
    aspects_list: str = dspy.OutputField(desc=dedent("""
        The value should be a valid Python list that contains all the aspects.
        """))

In [78]:
class Aspect2Reason(dspy.Signature):
    """
    Using the customer review and given aspect, reason about whether the aspect is positive, neutral, or negative.
    Think deeply about the aspect in the context of the entire review.
    """

    review: str = dspy.InputField(desc="a customer review")
    aspect: str = dspy.InputField(desc=dedent("""
        An aspect that is mentioned in the customer review
        """))
    reason: str = dspy.OutputField(desc=dedent("""
        Reasons why the given aspect is positively described, neutrally described, or negatively described in the customer review.
        """))

In [79]:
class Reason2Label(dspy.Signature):
    """
    Looks at the reason for whether the given aspect is positive, neutral, or negative, and labels it with a
    """

    aspect: str = dspy.InputField(desc=dedent("""
        An aspect that is mentioned in the customer review
        """))
    reason: str = dspy.InputField(desc=dedent("""
        Reasons why the given aspect is positively described, neutrally described, or negatively described in a customer review.
        """))
    label: int = dspy.OutputField(format=int, desc=dedent("""
        an integer. 1 indicates positive sentiment. 0 indicates neutral sentiment. -1 indicates negative sentiment.
        """))

In [80]:
import ast

class ABSAPipelineV3(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_aspects = dspy.ChainOfThought(Review2AspectsList)
        self.generate_reasons = dspy.ChainOfThought(Aspect2Reason)
        self.generate_label = dspy.TypedChainOfThought(Reason2Label)

    def forward(self, review):
        aspects = self.generate_aspects(review=review)
        aspects_with_label = {}
        for aspect in ast.literal_eval(aspects.aspects_list):
            if len(aspect) == 1:
                print("invalid aspect")
                continue
            reason = self.generate_reasons(review=review, aspect=aspect)
            label = self.generate_label(aspect=aspect, reason=reason.reason)
            aspects_with_label[aspect] = int(label.label)
        return dspy.Prediction(aspects_with_label=aspects_with_label)

In [75]:
uncompiled_pipeline_v3 = ABSAPipelineV3()
pred = uncompiled_pipeline_v3(dspy_test[0].review)
pred.aspects_with_label

{'menu': 1,
 'pub fare': 0,
 'burgers': 1,
 'steaks': 1,
 'shepherds pie': 1,
 'portabella lasagna': 1,
 'vegetarians': 0}

In [30]:
 evaluate_test(uncompiled_pipeline_v3, metric=validate_f1)


Average Metric: 0.5 / 6  (8.3):   5%|▊              | 5/100 [00:03<01:28,  1.07it/s]

Error for example in dev set: 		 ('Too many retries trying to get the correct output format. Try simplifying the requirements.', {'aspects': 'Field required: value (error type: missing)'})



Average Metric: 0.8333333333333333 / 7  (11.9):   6%| | 6/100 [00:04<01:27,  1.07it/
Average Metric: 0.8333333333333333 / 7  (11.9):   7%| | 7/100 [00:04<00:51,  1.82it/
Average Metric: 0.8333333333333333 / 8  (10.4):   7%| | 7/100 [00:04<00:51,  1.82it/
Average Metric: 0.8333333333333333 / 8  (10.4):   8%| | 8/100 [00:04<00:41,  2.19it/
Average Metric: 1.6904761904761902 / 9  (18.8):   8%| | 8/100 [00:04<00:41,  2.19it/
Average Metric: 1.6904761904761902 / 9  (18.8):   9%| | 9/100 [00:04<00:38,  2.36it/
Average Metric: 1.6904761904761902 / 10  (16.9):   9%| | 9/100 [00:05<00:38,  2.36it
Average Metric: 1.6904761904761902 / 10  (16.9):  10%| | 10/100 [00:05<00:41,  2.15i
Average Metric: 2.6904761904761902 / 11  (24.5):  10%| | 10/100 [00:05<00:41,  2.15i
Average Metric: 2.6904761904761902 / 11  (24.5):  11%| | 11/100 [00:05<00:33,  2.65i

Error for example in dev set: 		 ('Too many retries trying to get the correct output format. Try simplifying the requirements.', {'aspects': 'Field required: value (error type: missing)'})



Average Metric: 2.6904761904761902 / 12  (22.4):  11%| | 11/100 [00:05<00:33,  2.65i
Average Metric: 2.6904761904761902 / 12  (22.4):  12%| | 12/100 [00:05<00:28,  3.04i
Average Metric: 3.6904761904761902 / 13  (28.4):  12%| | 12/100 [00:06<00:28,  3.04i
Average Metric: 3.6904761904761902 / 13  (28.4):  13%|▏| 13/100 [00:06<00:25,  3.43i
Average Metric: 3.6904761904761902 / 14  (26.4):  13%|▏| 13/100 [00:06<00:25,  3.43i
Average Metric: 3.6904761904761902 / 14  (26.4):  14%|▏| 14/100 [00:06<00:26,  3.26i
Average Metric: 3.6904761904761902 / 15  (24.6):  14%|▏| 14/100 [00:06<00:26,  3.26i
Average Metric: 3.6904761904761902 / 15  (24.6):  15%|▏| 15/100 [00:06<00:27,  3.14i
Average Metric: 3.6904761904761902 / 16  (23.1):  15%|▏| 15/100 [00:07<00:27,  3.14i
Average Metric: 3.6904761904761902 / 16  (23.1):  16%|▏| 16/100 [00:07<00:27,  3.08i

...


Average Metric: 31.012698412698416 / 96  (32.3):  95%|▉| 95/100 [00:37<00:03,  1.50i
Average Metric: 31.012698412698416 / 96  (32.3):  96%|▉| 96/100 [00:37<00:02,  1.67i
Average Metric: 31.012698412698416 / 97  (32.0):  96%|▉| 96/100 [00:38<00:02,  1.67i
Average Metric: 31.012698412698416 / 97  (32.0):  97%|▉| 97/100 [00:38<00:01,  1.80i
Average Metric: 31.012698412698416 / 98  (31.6):  97%|▉| 97/100 [00:38<00:01,  1.80i
Average Metric: 31.262698412698416 / 99  (31.6):  98%|▉| 98/100 [00:39<00:01,  1.80i
Average Metric: 31.262698412698416 / 99  (31.6):  99%|▉| 99/100 [00:39<00:00,  1.95i
Average Metric: 31.929365079365084 / 100  (31.9):  99%|▉| 99/100 [00:40<00:00,  1.95
Average Metric: 31.929365079365084 / 100  (31.9): 100%|█| 100/100 [00:40<00:00,  2.4

Average Metric: 31.929365079365084 / 100  (31.9%)



/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)
/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:263: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.0' '0.0' '0.0' '0.0' '0.5']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,review,example_aspects,pred_aspects,validate_f1,aspects
0,""" The menu includes pub fare--burgers, steaks and shepherds pie--and even a portabella lasagna for those black sheep known as ""vegetarians.","{'menu': 0, 'burgers': 0, 'steaks': 0, 'shepherds pie': 0, 'portabella lasagna': 0, 'pub fare': 0}","{'menu': 1, 'pub fare': 1, 'burgers': 1, 'steaks': 0, 'shepherds pie': 1, 'portabella lasagna': 1, 'vegetarians': 0}",0.0,nan
1,(The sashimi is cut a little thinly.,{'sashimi': -1},"{'sashimi': -1, 'cut': -1, 'thinly': -1}",0.0,nan
2,") It's not the best Japanese restaurant in the East Village, but it's a pretty solid one for its modest prices, and worth repeat visits.",nan,nan,0.0,{'prices': 1}
3,15% gratuity automatically added to the bill.,"{'bill': -1, 'gratuity': -1}","{'gratuity': -1, 'bill': 0}",0.0,nan
4,"A beautiful atmosphere, perfect for drinks and/or appetizers.","{'atmosphere': 1, 'drinks': 0, 'appetizers': 0}","{'atmosphere': 1, 'drinks': 1, 'appetizers': 1}",0.5,nan


31.93

In [83]:
from dspy.teleprompt import BootstrapFewShotWithRandomSearch

optimizer = BootstrapFewShotWithRandomSearch(metric=validate_venn)
compiled_pipeline_v3 = optimizer.compile(ABSAPipelineV3(), teacher=ABSAPipelineV3(), trainset=dspy_train[:30])

Going to sample between 1 and 4 traces per predictor.
Will attempt to train 16 candidate sets.



Average Metric: 2.75 / 11  (25.0):  33%|████        | 10/30 [00:01<00:02,  7.40it/s]
Average Metric: 3.0833333333333335 / 12  (25.7):  37%|▎| 11/30 [00:01<00:02,  7.40it
Average Metric: 4.083333333333334 / 13  (31.4):  40%|▍| 12/30 [00:01<00:02,  7.40it/
Average Metric: 4.083333333333334 / 13  (31.4):  43%|▍| 13/30 [00:01<00:01, 11.86it/
Average Metric: 5.083333333333334 / 14  (36.3):  43%|▍| 13/30 [00:01<00:01, 11.86it/
Average Metric: 5.183333333333334 / 15  (34.6):  47%|▍| 14/30 [00:01<00:01, 11.86it/
Average Metric: 5.933333333333334 / 16  (37.1):  50%|▌| 15/30 [00:01<00:01, 11.86it/
Average Metric: 6.266666666666667 / 17  (36.9):  53%|▌| 16/30 [00:01<00:01, 11.86it/
Average Metric: 6.266666666666667 / 17  (36.9):  57%|▌| 17/30 [00:01<00:00, 15.18it/
Average Metric: 6.6 / 19  (34.7):  60%|███████▊     | 18/30 [00:01<00:00, 15.18it/s]
Average Metric: 6.933333333333333 / 20  (34.7):  63%|▋| 19/30 [00:01<00:00, 15.18it/
Average Metric: 7.266666666666666 / 21  (34.6):  67%|▋| 20/30 [0

Average Metric: 11.266666666666666 / 30  (37.6%)
Score: 37.56 for set: [0, 0, 0]
New best score: 37.56 for seed -3
Scores so far: [37.56]
Best score: 37.56



Average Metric: 0.5 / 2  (25.0):   7%|█              | 2/30 [00:01<00:13,  2.09it/s]
Average Metric: 0.8333333333333333 / 3  (27.8):   7%| | 2/30 [00:01<00:13,  2.09it/s
Average Metric: 1.8333333333333333 / 4  (45.8):  10%| | 3/30 [00:01<00:12,  2.09it/s
Average Metric: 1.8333333333333333 / 4  (45.8):  13%|▏| 4/30 [00:01<00:06,  4.10it/s
Average Metric: 2.733333333333333 / 6  (45.6):  20%|▏| 6/30 [00:01<00:03,  6.29it/s]
Average Metric: 3.3999999999999995 / 7  (48.6):  20%|▏| 6/30 [00:02<00:03,  6.29it/s
Average Metric: 3.8999999999999995 / 8  (48.7):  23%|▏| 7/30 [00:02<00:03,  6.29it/s
Average Metric: 3.8999999999999995 / 8  (48.7):  27%|▎| 8/30 [00:02<00:06,  3.18it/s
Average Metric: 5.5 / 12  (45.8):  40%|█████▏       | 12/30 [00:03<00:06,  2.95it/s]
Average Metric: 5.833333333333333 / 13  (44.9):  40%|▍| 12/30 [00:04<00:06,  2.95it/
Average Metric: 5.833333333333333 / 13  (44.9):  43%|▍| 13/30 [00:04<00:05,  3.31it/
Average Metric: 6.583333333333333 / 14  (47.0):  43%|▍| 13/30 [0

Average Metric: 17.0 / 30  (56.7%)
Score: 56.67 for set: [16, 16, 16]
New best score: 56.67 for seed -2
Scores so far: [37.56, 56.67]
Best score: 56.67


...


Average Metric: 11.958333333333332 / 24  (49.8):  77%|▊| 23/30 [01:22<00:19,  2.76s/
Average Metric: 11.958333333333332 / 24  (49.8):  80%|▊| 24/30 [01:22<00:14,  2.42s/
Average Metric: 12.958333333333332 / 25  (51.8):  80%|▊| 24/30 [01:23<00:14,  2.42s/
Average Metric: 12.958333333333332 / 25  (51.8):  83%|▊| 25/30 [01:23<00:09,  1.93s/

Backing off 0.5 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}



Average Metric: 12.958333333333332 / 26  (49.8):  83%|▊| 25/30 [01:30<00:09,  1.93s/
Average Metric: 12.958333333333332 / 26  (49.8):  87%|▊| 26/30 [01:30<00:14,  3.51s/
Average Metric: 13.624999999999998 / 27  (50.5):  87%|▊| 26/30 [01:31<00:14,  3.51s/
Average Metric: 13.624999999999998 / 27  (50.5):  90%|▉| 27/30 [01:31<00:08,  2.76s/
Average Metric: 14.624999999999998 / 28  (52.2):  90%|▉| 27/30 [01:33<00:08,  2.76s/
Average Metric: 14.624999999999998 / 28  (52.2):  93%|▉| 28/30 [01:33<00:05,  2.55s/
Average Metric: 15.374999999999998 / 29  (53.0):  93%|▉| 28/30 [01:38<00:05,  2.55s/
Average Metric: 15.374999999999998 / 29  (53.0):  97%|▉| 29/30 [01:38<00:03,  3.26s/
Average Metric: 15.708333333333332 / 30  (52.4):  97%|▉| 29/30 [01:52<00:03,  3.26s/
Average Metric: 15.708333333333332 / 30  (52.4): 100%|█| 30/30 [01:52<00:00,  3.73s/

Average Metric: 15.708333333333332 / 30  (52.4%)
Score: 52.36 for set: [16, 16, 16]
Scores so far: [37.56, 56.67, 47.49, 51.53, 46.28, 35.64, 50.61, 51.97, 45.64, 49.44, 46.62, 48.28, 51.72, 51.38, 51.77, 43.0, 39.19, 45.67, 52.36]
Best score: 56.67
Average of max per entry across top 1 scores: 0.5666666666666667
Average of max per entry across top 2 scores: 0.7583333333333332
Average of max per entry across top 3 scores: 0.8547222222222223
Average of max per entry across top 5 scores: 0.8927777777777778
Average of max per entry across top 8 scores: 0.9683333333333334
Average of max per entry across top 9999 scores: 1.0
19 candidate programs found.



/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)


In [86]:
 evaluate_test(compiled_pipeline_v3, metric=validate_f1)



  0%|                                                       | 0/100 [00:00<?, ?it/s]

Average Metric: 0 / 1  (0.0):   0%|                         | 0/100 [00:00<?, ?it/s]

Average Metric: 0 / 1  (0.0):   1%|▏                | 1/100 [00:00<00:12,  7.71it/s]

Average Metric: 0.0 / 2  (0.0):   1%|▏              | 1/100 [00:00<00:12,  7.71it/s]

Average Metric: 0.0 / 2  (0.0):   2%|▎              | 2/100 [00:00<00:11,  8.30it/s]

Average Metric: 0.0 / 3  (0.0):   2%|▎              | 2/100 [00:00<00:11,  8.30it/s]

Average Metric: 0.0 / 4  (0.0):   3%|▍              | 3/100 [00:00<00:11,  8.30it/s]

Average Metric: 0.5 / 5  (10.0):   4%|▌             | 4/100 [00:00<00:11,  8.30it/s]

Average Metric: 1.1666666666666665 / 6  (19.4):   5%| | 5/100 [00:00<00:11,  8.30it/

Average Metric: 1.1666666666666665 / 6  (19.4):   6%| | 6/100 [00:00<00:05, 15.76it/

Average Metric: 1.5666666666666664 / 7  (22.4):   6%| | 6/100 [00:00<00:05, 15.76it/

Average Metric: 2.5666666666666664 / 8  (32.1):   7%

Backing off 0.4 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 0.2 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 0.5 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 0.2 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {'max_tokens': 500, 'n': 1, 'temperature': 0.0}
Backing off 0.7 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {'max_tokens': 500, 'n': 1, 'temperature': 0.0}




Average Metric: 29.969047619047625 / 70  (42.8):  69%|▋| 69/100 [00:04<00:00, 35.45i

Backing off 0.8 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 0.2 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 0.0 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 1.6 seconds after 2 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {'max_tokens': 500, 'n': 1, 'temperature': 0.0}
Backing off 0.2 seconds after 2 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 0.9 seconds after 1 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 0.5 seconds after 2 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {}
Backing off 0.5 seconds after 2 tries calling function <function GPT3.request at 0x3106b0e00> with kwargs {'max_tokens': 500, 'n': 1, 'temperature': 0.0}




Average Metric: 29.969047619047625 / 71  (42.2):  70%|▋| 70/100 [00:07<00:00, 35.45i

...



Average Metric: 42.82619047619046 / 93  (46.0):  92%|▉| 92/100 [01:12<00:22,  2.86s/

Average Metric: 42.82619047619046 / 93  (46.0):  93%|▉| 93/100 [01:12<00:19,  2.81s/

Average Metric: 42.82619047619046 / 94  (45.6):  93%|▉| 93/100 [01:13<00:19,  2.81s/

Average Metric: 42.82619047619046 / 94  (45.6):  94%|▉| 94/100 [01:13<00:13,  2.30s/

Average Metric: 42.82619047619046 / 95  (45.1):  94%|▉| 94/100 [01:14<00:13,  2.30s/

Average Metric: 42.82619047619046 / 95  (45.1):  95%|▉| 95/100 [01:14<00:08,  1.76s/

Average Metric: 42.82619047619046 / 96  (44.6):  95%|▉| 95/100 [01:15<00:08,  1.76s/

Average Metric: 42.82619047619046 / 96  (44.6):  96%|▉| 96/100 [01:15<00:06,  1.72s/

Average Metric: 42.82619047619046 / 97  (44.2):  96%|▉| 96/100 [01:23<00:06,  1.72s/

Average Metric: 42.82619047619046 / 97  (44.2):  97%|▉| 97/100 [01:23<00:10,  3.62s/

Average Metric: 43.82619047619046 / 98  (44.7):  97%|▉| 97/100 [01:59<00:10,  3.62s/

Average Metric: 43.82619047619046 / 98  (44.7):  98%

Average Metric: 44.32619047619046 / 100  (44.3%)



/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:187: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(truncate_cell)
/Users/richy/.pyenv/versions/3.11.7/envs/llm/lib/python3.11/site-packages/dspy/evaluate/evaluate.py:263: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '['0.0' '0.0' '0.6666666666666666' '0.0' '0.5']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[:, metric_name] = df[metric_name].apply(


,review,example_aspects_with_label,aspects_list,pred_aspects_with_label,validate_f1
0,""" The menu includes pub fare--burgers, steaks and shepherds pie--and even a portabella lasagna for those black sheep known as ""vegetarians.","{'menu': 0, 'burgers': 0, 'steaks': 0, 'shepherds pie': 0, 'portabella lasagna': 0, 'pub fare': 0}","dict_keys(['menu', 'burgers', 'steaks', 'shepherds pie', 'portabella lasagna', 'pub fare'])","{'menu': 1, 'pub fare': 0, 'burgers': 1, 'steaks': 1, 'shepherds pie': 1, 'portabella lasagna': 1, 'vegetarians': 0}",0.0
1,(The sashimi is cut a little thinly.,{'sashimi': -1},dict_keys(['sashimi']),"{'sashimi': -1, 'thinly': -1}",0.0
2,") It's not the best Japanese restaurant in the East Village, but it's a pretty solid one for its modest prices, and worth repeat visits.",{'prices': 1},dict_keys(['prices']),"{'Japanese restaurant': 0, 'East Village': 0, 'prices': 1, 'repeat visits': 1}",0.6666666666666666
3,15% gratuity automatically added to the bill.,"{'bill': -1, 'gratuity': -1}","dict_keys(['bill', 'gratuity'])","{'gratuity': -1, 'bill': -1}",0.0
4,"A beautiful atmosphere, perfect for drinks and/or appetizers.","{'atmosphere': 1, 'drinks': 0, 'appetizers': 0}","dict_keys(['atmosphere', 'drinks', 'appetizers'])","{'atmosphere': 1, 'drinks': 1, 'appetizers': 1}",0.5


44.33

In [87]:
compiled_pipeline_v3.save("./ABSAPipelineV3venn.json")

In [ ]:
v3 = ABSAPipelineV3()
v3_venn = ABSAPipelineV3()
v3 = ABSAPipelineV3.load("./ABSAPipelineV3venn.json")